Load the CSV file into a Data Frame object named "chatterbox."

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
chatterbox = pd.read_csv("employees.csv")

Identify the data type of each variable.

In [3]:
features = chatterbox.columns

In [4]:
# chatterbox.info

In [5]:
# chatterbox.isna().sum()

In [6]:
(chatterbox['Employee_No'] == chatterbox['Reporting_emp_1'])

In [7]:
chatterbox.loc[(chatterbox['Employee_No'] == chatterbox['Reporting_emp_1']) , 'Reporting_emp_1'] = np.nan

In [8]:
chatterbox.loc[((chatterbox['Date_Resigned'] == r'\N') | (chatterbox['Date_Resigned'] == r'0000-00-00')) , 'Date_Resigned'] = chatterbox['Inactive_Date']

chatterbox.loc[(chatterbox['Date_Resigned'] == r'0000-00-00') & (chatterbox['Status'] == r'Active') , 'Date_Resigned'] = r'\N'
chatterbox.loc[(chatterbox['Inactive_Date'] == r'0000-00-00') & (chatterbox['Status'] == r'Active') , 'Inactive_Date'] = r'\N'

In [9]:
chatterbox.loc[((chatterbox['Title'] == r'Mr') | (chatterbox['Title'] == r'Miss')) & (chatterbox['Gender'] == r'Female') , 'Title'] = r'Ms'

chatterbox.loc[((chatterbox['Title'] == r'Ms') | (chatterbox['Title'] == r'Miss')) & (chatterbox['Gender'] == r'Male') , 'Title'] = r'Mr'

In [10]:
chatterbox['Year_of_Birth'] = pd.to_numeric(chatterbox['Year_of_Birth'], errors='coerce')

In [11]:
chatterbox['Year_Joined'] = pd.to_datetime(chatterbox['Date_Joined']).dt.year

chatterbox['Experience'] = 2022 - chatterbox['Year_Joined']

In [12]:
chatterbox.loc[chatterbox['Year_of_Birth'] == '0000', 'Year_of_Birth'] = np.nan

In [13]:
# Split the data into two sets: one with missing values and the other with non-missing values
missing_values = chatterbox[chatterbox['Year_of_Birth'].isnull()]
non_missing_values = chatterbox[chatterbox['Year_of_Birth'].notnull()]

# Prepare the features and target for training
non_missing_features = non_missing_values[['Experience', 'Designation','Marital_Status']]
target = non_missing_values['Year_of_Birth']

# Encode the 'Designation' column using one-hot encoding with handle_unknown='ignore'
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(non_missing_features[['Designation','Marital_Status']])

# Train a Decision Tree classifier model
model = DecisionTreeClassifier()
model.fit(encoded_features.toarray(), target)

# Encode the missing values for prediction
missing_features = missing_values[['Year_Joined', 'Designation','Marital_Status']]
encoded_missing_features = encoder.transform(missing_features[['Designation','Marital_Status']])

# Predict the missing values using the trained model
predicted_values = model.predict(encoded_missing_features.toarray())

# Fill in the missing values in the 'Year_of_Birth' column with the predicted values
chatterbox.loc[chatterbox['Year_of_Birth'].isnull(), 'Year_of_Birth'] = np.round(predicted_values)

# Convert 'Year_of_Birth' column back to integer type
chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].astype(int)

# Replace '0000' values with NaN
# chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].replace(0, np.nan)

# Print the updated DataFrame
# print(chatterbox)


In [14]:
# fig, ax = plt.subplots(figsize=(14, 6))

# # Create a count plot of marital status categories
# sns.countplot(data=chatterbox, x='Age', hue='Marital_Status')
# plt.xlabel('Age')
# plt.ylabel('Count')
# plt.title('Marital Status Distribution by Age')
# plt.xticks(rotation=90)
# plt.show()

In [15]:
# # Create a scatter plot of age vs. year joined
# plt.scatter(chatterbox['Experience'], chatterbox['Age'])
# plt.xlabel('Experience')
# plt.ylabel('Age')
# plt.title('Age vs. Year Joined')
# plt.show()

In [16]:
# chatterbox.isna().sum()

In [17]:
# chatterbox['Marital_Status11'] = chatterbox['Marital_Status']

In [18]:
# Split the data into two sets: one with missing values and the other with non-missing values
missing_values = chatterbox[chatterbox['Marital_Status'].isnull()]
non_missing_values = chatterbox[chatterbox['Marital_Status'].notnull()]

# Prepare the features and target for training
non_missing_features = non_missing_values[['Year_of_Birth','Gender']]
target = non_missing_values['Marital_Status']

# Encode the 'Designation' column using one-hot encoding with handle_unknown='ignore'
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(non_missing_features[['Year_of_Birth','Gender']])

# Train a Decision Tree classifier model
model = DecisionTreeClassifier()
model.fit(encoded_features.toarray(), target)

# Encode the missing values for prediction
missing_features = missing_values[['Year_of_Birth','Gender']]
encoded_missing_features = encoder.transform(missing_features[['Year_of_Birth','Gender']])

# Predict the missing values using the trained model
predicted_values = model.predict(encoded_missing_features.toarray())

# Fill in the missing values in the 'Status' column with the predicted values
chatterbox.loc[chatterbox['Marital_Status'].isnull(), 'Marital_Status'] = predicted_values

# Convert 'Status' column back to string type
chatterbox['Marital_Status'] = chatterbox['Marital_Status'].astype(str)

In [19]:
# Step 4: Identify and handle data quality issues
chatterbox.drop_duplicates(inplace=True)

In [20]:
# chatterbox.to_csv('check.csv', index=False)

In [21]:
final = chatterbox[features]

In [22]:
final.loc[final['Date_Resigned'] == r'\N', 'Date_Resigned'] = np.nan

final.loc[final['Inactive_Date'] == r'\N', 'Inactive_Date'] = np.nan

final.loc[final['Reporting_emp_1'] == r'\N', 'Reporting_emp_1'] = np.nan

final.loc[final['Reporting_emp_2'] == r'\N', 'Reporting_emp_2'] = np.nan

In [23]:
# final['Date_Resigned'] = pd.to_datetime(final['Date_Resigned'])

In [24]:
# final['Date_Resigned'].unique()

In [25]:
final.to_csv('employee_preprocess_200647R.csv', index=False)